In [ ]:
# Run this to install all the necessary packages

!pip install numpy torch scikit-learn requests matplotlib

In [ ]:
# Import all the required packages

import os
import torch
import requests
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# Prepare dataset for training

def prepare_dataset(sequence_length=20):
    # Download the first 100,000 digits of Pi
    url = "https://www.angio.net/pi/digits/100000.txt"
    response = requests.get(url)
    pi_digits = response.text.replace("\n", "").replace(" ", "")

    # Remove "3." at the beginning of Pi
    pi_digits = pi_digits[2:]

    # Convert digits into numerical format
    digits = np.array([int(d) for d in pi_digits])

    # Create sequences and corresponding labels
    X, y = [], []
    for i in range(len(digits) - sequence_length):
        X.append(digits[i : i + sequence_length])
        y.append(digits[i + sequence_length])

    # Convert to numpy arrays
    X = np.array(X) / 9.0  # Normalize input (0 to 1)
    y = np.array(y)  # Target is a digit (0-9)

    # Convert to PyTorch tensors
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)

    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

    print("💾 Dataset prepared and stored in memory!")
    return X_train, X_test, y_train, y_test

# Call the function and store the dataset in memory
X_train, X_test, y_train, y_test = prepare_dataset()

In [ ]:
# Define LSTM Model 

class PiPredictorLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=10):
        super(PiPredictorLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x.unsqueeze(-1), (h0, c0))
        out = self.fc(out[:, -1, :])  # Get output from last time step

        return out

In [ ]:
# Train the model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PiPredictorLSTM().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create DataLoader
batch_size = 64
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)

# Train model (No Saving to Disk)
num_epochs = 10
train_losses = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("✅ Pi Model trained and stored in memory!")

In [ ]:
# Evaluate model performance

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for i in range(len(X_test)):
        X = X_test[i].unsqueeze(0).to(device)
        y = y_test[i].to(device)

        outputs = model(X)
        _, predicted = torch.max(outputs, 1)

        correct += (predicted == y).sum().item()
        total += 1

accuracy = correct / total
print(f"📊 Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Compare model performance with random guessing

random_accuracy = sum(random.randint(0, 9) == y.item() for y in y_test) / len(y_test)
print(f"🎲 Random Guessing Accuracy: {random_accuracy * 100:.2f}%")

In [ ]:
# Predict the next digit given the first 20 digits (This is for you to try by giving input)

def predict_next_digit(input_digits):
    # Remove non-digit characters (like ".")
    clean_digits = [d for d in input_digits if d.isdigit()]
    
    # Convert to numerical format
    digits = np.array([int(d) for d in clean_digits])

    # Check if input length is valid
    sequence_length = 20
    if len(digits) < sequence_length:
        print(f"⚠️ Input must be at least {sequence_length} digits long!")
        return
    
    # Use the last `sequence_length` digits
    digits = digits[-sequence_length:]

    # Normalize input (scale between 0 and 1)
    X_input = np.array(digits) / 9.0
    X_tensor = torch.tensor(X_input, dtype=torch.float32).unsqueeze(0).to(device)

    # Predict the next digit
    with torch.no_grad():
        output = model(X_tensor)
        predicted_digit = torch.argmax(output, dim=1).item()

    print(f"🔢 Predicted next digit after {input_digits} is: {predicted_digit}")

# Get user input and predict
user_input = input("Enter a sequence of Pi digits: ")
predict_next_digit(user_input)